In [1]:
#!pip install jpstat geojson folium shapely plotly topojson chardet geopandas pytopojson matplotlib folium

import geopandas as gpd
import topojson as tp
import jpstat
import pandas as pd
#import geojson
import requests
#import folium
import plotly.express as px
#from shapely.geometry import shape
#from urllib.request import urlopen
import json
from pytopojson import topology, feature
import matplotlib.pyplot as plt
import folium

In [2]:
stat = jpstat.estat.get_stat(key='9172300186e42e3c1e0acd0ce205021a2f27f28b')
jpstat.options["estat.api_key"] = '9172300186e42e3c1e0acd0ce205021a2f27f28b'

In [3]:
data, note = jpstat.estat.get_data(statsDataId="0004003462")
#data.sample(n=4)
#note

..

In [4]:
data = data.loc[(data["国籍"] == "移動者") & (data["移動前の住所地（前住地）2020～"] == "総数(前住地)")]

In [5]:
pref = pd.DataFrame(data={'En': ['Hokkaido', 'Aomori', 'Iwate', 'Miyagi', 'Akita', 'Yamagata', 'Fukushima', 'Ibaraki', 'Tochigi', 'Gunma', 'Saitama', 
                                 'Chiba', 'Tokyo', 'Kanagawa', 'Niigata', 'Toyama', 'Ishikawa', 'Fukui', 'Yamanashi', 'Nagano', 'Gifu', 'Shizuoka', 
                                 'Aichi', 'Mie', 'Shiga', 'Kyoto', 'Osaka', 'Hyogo', 'Nara', 'Wakayama', 'Tottori', 'Shimane', 'Okayama', 'Hiroshima', 
                                 'Yamaguchi', 'Tokushima', 'Kagawa', 'Ehime', 'Kochi', 'Fukuoka', 'Saga', 'Nagasaki', 'Kumamoto', 'Oita', 'Miyazaki', 
                                 'Kagoshima', 'Okinawa'], 
                          'Ja': ['北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県', '福島県', '茨城県', '栃木県', '群馬県', '埼玉県', '千葉県', 
                                 '東京都', '神奈川県', '新潟県', '富山県', '石川県', '福井県', '山梨県', '長野県', '岐阜県', '静岡県', '愛知県', '三重県', 
                                 '滋賀県', '京都府', '大阪府', '兵庫県', '奈良県', '和歌山県', '鳥取県', '島根県', '岡山県', '広島県', '山口県', '徳島県', 
                                 '香川県', '愛媛県', '高知県', '福岡県', '佐賀県', '長崎県', '熊本県', '大分県', '宮崎県', '鹿児島県', '沖縄県'], 
                          'ISO': ['JP-01', 'JP-02', 'JP-03', 'JP-04', 'JP-05', 'JP-06', 'JP-07', 'JP-08', 'JP-09', 'JP-10', 'JP-11', 'JP-12', 'JP-13', 
                                  'JP-14', 'JP-15', 'JP-16', 'JP-17', 'JP-18', 'JP-19', 'JP-20', 'JP-21', 'JP-22', 'JP-23', 'JP-24', 'JP-25', 'JP-26', 
                                  'JP-27', 'JP-28', 'JP-29', 'JP-30', 'JP-31', 'JP-32', 'JP-33', 'JP-34', 'JP-35', 'JP-36', 'JP-37', 'JP-38', 'JP-39', 
                                  'JP-40', 'JP-41', 'JP-42', 'JP-43', 'JP-44', 'JP-45', 'JP-46', 'JP-47'],
                          'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 
                                 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]})

joined_data = data.join(pref.set_index('Ja'), on='移動後の住所地（現住地）2020～', how='right')
#data['移動後の住所地（現住地）2020～']['香川県']
#data.filter(items=pref.keys())
#type(joined_data["id"][0])

In [6]:
# コロプレス図の中心
CENTER = {'lat': 36, 'lon': 140}

In [10]:
#japan_geo = requests.get("https://raw.githubusercontent.com/dataofjapan/land/master/japan.topojson").json()
#m = folium.Map(location=[36, 138], zoom_start=5)

prefectures = requests.get(
    'https://raw.githubusercontent.com/dataofjapan/land/92934a149860bc23b529ccbeba1e8366b30b0c63/japan.geojson')

jpn = gpd.read_file(prefectures.text)

In [11]:
jpn.set_index('ID', drop=False, inplace=True)
  
print(jpn.info())
display(jpn.head())

KeyError: "None of ['ID'] are in the columns"

In [94]:
joined_data['id'] = joined_data['id'].astype(int)
type(joined_data["id"][0])

numpy.int32

In [96]:
japan_geo_ids = [geo["properties"]["id"] for geo in japan_geo["objects"]["japan"]["geometries"]]

In [105]:
is_in_joined_data = joined_data["id"].isin(japan_geo_ids)
not_in_joined_data = joined_data[~is_in_joined_data]
print(not_in_joined_data)

Empty DataFrame
Columns: [@unit, 表章項目, 移動後の住所地（現住地）2020～, 国籍, 移動前の住所地（前住地）2020～, 時間軸, Value, En, ISO, id]
Index: []


In [114]:
print(japan_geo["transform"].keys())

dict_keys(['scale', 'translate'])


In [95]:
folium.Choropleth(
    geo_data=japan_geo,
    name="choropleth",
    data=joined_data,
    columns=["移動後の住所地（現住地）2020～", "Value"],
    topojson="objects.japan",
    key_on="feature.properties.nam_ja",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Number of People Who Moved to Location").add_to(m)

folium.LayerControl().add_to(m)

m

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [18]:
# Fetch the GeoJSON content from the URL
#prefectures = requests.get(
#    'https://raw.githubusercontent.com/dataofjapan/land/92934a149860bc23b529ccbeba1e8366b30b0c63/japan.geojson')

#prefectures =  geojson.loads(prefectures.text)

#japan_geojson_url = 'https://raw.githubusercontent.com/dataofjapan/land/92934a149860bc23b529ccbeba1e8366b30b0c63/japan.geojson'
#japan_geojson = geojson.loads(requests.get(japan_geojson_url).text)

# Fetch the TopoJSON content from the URL
japan_topojson_url = 'https://raw.githubusercontent.com/dataofjapan/land/master/japan.topojson'
japan_topojson_request = requests.get(japan_topojson_url).json()

In [20]:
# Convert TopoJSON to GeoJSON
japan_geojson = topojson.feature(japan_topojson_request)

AttributeError: module 'topojson' has no attribute 'feature'

In [6]:
# Convert TopoJSON to GeoJSON
japan_topojson = tp.topology(japan_topojson_request)

features = japan_topojson['features']
ids = sorted(set([feature['properties']['id'] for feature in features]))

new_features = []

for id in ids:
    data = [feature['geometry']['coordinates'][0][0] for feature in features if feature['properties']['id'] == id]
    
    if len(data) == 1:
        feature = dict(type="Feature", geometry=dict(type="Polygon", coordinates=data[0]), properties=dict(id=id))
    else:
        feature = dict(type="Feature", geometry=dict(type="MultiPolygon", coordinates=data), properties=dict(id=id))
    
    new_features.append(feature)

japan_topojson['features'] = new_features

AttributeError: module 'topojson' has no attribute 'topology'

In [7]:
print(data)

data


In [ ]:
# Load the GeoJSON content
geojson_data = geojson.loads(prefectures.text)

# Initialize an empty list to store all the coordinates
all_coordinates = []

# Extract coordinates from each feature
for feature in geojson_data['features']:
    coordinates = feature['geometry']['coordinates'][0]
    all_coordinates.extend(coordinates)

# Calculate the centroid of all coordinates
centroid = shape({'type': 'Polygon', 'coordinates': [all_coordinates]}).centroid
#center = [centroid.y, centroid.x]

# Create a folium map centered on the calculated centroid
#my_map = folium.Map(location=center, zoom_start=10)

# Add GeoJSON features to the map
#folium.GeoJson(geojson_data).add_to(my_map)

# Display the map in Jupyter Notebook
#my_map

In [ ]:
#corlen = 0
corloc = 0
for L in all_coordinates:
    corloc = corloc + 1
    if not isinstance(L[0], list):
        print(corloc)
        print(type(L[0]))
#print(corlen)    
#len(all_coordinates)[8710]

In [ ]:
pref = pd.DataFrame(data={'En': ['Hokkaido', 'Aomori', 'Iwate', 'Miyagi', 'Akita', 'Yamagata', 'Fukushima', 'Ibaraki', 'Tochigi', 'Gunma', 'Saitama', 
                                 'Chiba', 'Tokyo', 'Kanagawa', 'Niigata', 'Toyama', 'Ishikawa', 'Fukui', 'Yamanashi', 'Nagano', 'Gifu', 'Shizuoka', 
                                 'Aichi', 'Mie', 'Shiga', 'Kyoto', 'Osaka', 'Hyogo', 'Nara', 'Wakayama', 'Tottori', 'Shimane', 'Okayama', 'Hiroshima', 
                                 'Yamaguchi', 'Tokushima', 'Kagawa', 'Ehime', 'Kochi', 'Fukuoka', 'Saga', 'Nagasaki', 'Kumamoto', 'Oita', 'Miyazaki', 
                                 'Kagoshima', 'Okinawa'], 
                          'Ja': ['北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県', '福島県', '茨城県', '栃木県', '群馬県', '埼玉県', '千葉県', 
                                 '東京都', '神奈川県', '新潟県', '富山県', '石川県', '福井県', '山梨県', '長野県', '岐阜県', '静岡県', '愛知県', '三重県', 
                                 '滋賀県', '京都府', '大阪府', '兵庫県', '奈良県', '和歌山県', '鳥取県', '島根県', '岡山県', '広島県', '山口県', '徳島県', 
                                 '香川県', '愛媛県', '高知県', '福岡県', '佐賀県', '長崎県', '熊本県', '大分県', '宮崎県', '鹿児島県', '沖縄県'], 
                          'ISO': ['JP-01', 'JP-02', 'JP-03', 'JP-04', 'JP-05', 'JP-06', 'JP-07', 'JP-08', 'JP-09', 'JP-10', 'JP-11', 'JP-12', 'JP-13', 
                                  'JP-14', 'JP-15', 'JP-16', 'JP-17', 'JP-18', 'JP-19', 'JP-20', 'JP-21', 'JP-22', 'JP-23', 'JP-24', 'JP-25', 'JP-26', 
                                  'JP-27', 'JP-28', 'JP-29', 'JP-30', 'JP-31', 'JP-32', 'JP-33', 'JP-34', 'JP-35', 'JP-36', 'JP-37', 'JP-38', 'JP-39', 
                                  'JP-40', 'JP-41', 'JP-42', 'JP-43', 'JP-44', 'JP-45', 'JP-46', 'JP-47']})

joined_data = data.join(pref.set_index('Ja'), on='移動後の住所地（現住地）2020～', how='right')
#data['移動後の住所地（現住地）2020～']['香川県']
#data.filter(items=pref.keys())
joined_data.sample(n=10)

In [ ]:
features = japan_geojson['features']
ids = sorted(set([feature['properties']['id'] for feature in features]))

new_features = []

for id in ids:
    data = [feature['geometry']['coordinates'][0][0] for feature in features if feature['properties']['id'] == id]
    if len(data) == 1:
        feature = dict(type="Feature", geometry=dict(type="Polygon", coordinates=data[0]), properties=dict(id=id))
    else:
        feature = dict(type="Feature", geometry=dict(type="MultiPolygon", coordinates=data), properties=dict(id=id))
    new_features.append(feature)

japan_geojson['features'] = new_features

In [ ]:
fig = px.choropleth(joined_data, geojson=japan_geojson, color="Value", locations="ISO", title = 'n Moved to loc')
fig.update_geos(fitbounds="locations")
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

In [ ]:
fig.show()

In [ ]:
features = d['features']
ids = sorted(set([d['id'] for d in features]))
new_features = []
for id in ids:
    data = [d['geometry']['coordinates'] for d in features if d['id']==id]
    if len(data) == 1:
        feature = dict(type = "Feature", geometry = dict(type = "Polygon",coordinates = data[0]) id = id)
    else:
        feature = dict(type = "Feature", geometry = dict(type = "MultiPolygon", coordinates = data) id = id)
    new_features.append(feature)
d['features'] = new_features

#with open('new_tokyo23.json', 'w') as f:
#    json.dump(d, f)